In [1]:
from jupyter_dash import JupyterDash
import dash as dcc
import dash as html
from dash.dependencies import Input, Output

import plotly
from plotly.graph_objs import graph_objs as go

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import pickle
from skimage.filters import threshold_otsu
from scipy.ndimage import distance_transform_edt
from scipy.spatial import distance
from scipy.spatial import distance_matrix
import gudhi, gudhi.hera, gudhi.wasserstein, persim
import json

In [4]:
import mdtraj
from mdtraj import load

In [ ]:
app = JupyterDash(__name__)

## try to make this code look really similar to genAlphaSlider


### Vanilla version in topTraj and topMOF_pointcloud

### not putting it here because it's distracting and large



In [9]:
def pDistIndex(pdist,i: int, j: int,length: int):
    if j<i:
        i,j = j,i
    if i==j:
        return 0
    
    return length*i+j-((i+2)*(i+1)) //2


class dashTopViz():
    def __init__(self,dat,initial=0,step=.25,maximum=20,appName=None):
        if appName:
            self.app=JupyterDash(appName)
        else:
            self.app = JupyterDash(__name__)
        nD=dat.shape[1]
        
        assert nD<=3 #for now just implement xyz cords for polymer and mof ptCld
        ac=gudhi.AlphaComplex(dat)
        if not maximum:
            maximum=np.sum([(np.max(dat)-np.min(dat))**2 for i in range(nD)])
            maximum = np.sqrt(maximum)
        self.max=maximum
        st = ac.create_simplex_tree()#max_alpha_square=self.max**2)
        self.dgm=st.persistence()
        
        skel=list(st.get_skeleton(2))
        skel.sort(key=lambda s: s[1])
        points = np.array([ac.get_point(i) for i in range(st.num_vertices())]) # later change the mapping back to dat 
        
        dm=distance.pdist(points)
        skelD=[(i[0],dm[pDistIndex(dm,i[0][0],i[0][1],dat.shape[0])]) if len(i[0])==2 else (i[0],max({dm[pDistIndex(dm,i[0][k],i[0][j],dat.shape[0])] for k in range(len(i[0])) for j in range(k) })) for i in skel[len(points):]] # not true for 3simplices  
        skelD.sort(key=lambda s: s[1])
        
        self.alpha=initial
        
        self.alphaRes=step
        
        #make bins
        
        resIndex=1
        curdist=0
        skelBins=[]
        nextHigh=self.alphaRes
        #binLow=0
        while resIndex < len(skelD) and curdist<self.max:
            curdist=skelD[resIndex][1]
            if curdist > nextHigh:
                skelBins.append(resIndex) # up to but not including resIndex
                nextHigh+=self.alphaRes
            resIndex+=1
        if curdist<self.max:
            skelBins.append(len(skelD))
        print(len(skelD))
        print(skelBins[-5:])
        self.skel={k*self.alphaRes:[j[0] for j in skelD[:skelBins[k]]] for k in range(int(self.max//self.alphaRes))} 
        
        


def genAppFromXYZ(dat,initial=.05,step=.25,maximum=20):
    app = JupyterDash(__name__)

    app.layout = html.Div([
        html.H4('Simplicial Complex Explorer'),
        html.Div([
        "Filter points with persistence less than (enter number): ",
        dcc.Input(id='filterPH', value=0, type='number')
    ]),
        dcc.Graph(id='graph-with-slider'),
        
    ])
    
    
    
    return app



In [ ]:
skelD

In [5]:
os.chdir(r"Z:\data\micelles\Micelles_identification")
polyComp=os.listdir()
polyComp=[i for i in polyComp if i.find(".")==-1 and i!="High_conc"] #get rid of actionList if present -> do High_conc separately
temps=[int(i)*10 for i in range(1,6)]
polyComp

['L64', 'P104', 'P123', 'P65', 'P84', 'P85']

In [6]:
#Load a testdat
comp="P85"
temp=temps[0]
temp=30

groFile=mdtraj.load(f"{comp}/{comp}_{temp}.gro")

p0Index=[a.index for a in groFile.topology.atoms if a.name.find('PO')!=-1]
p0Gro=groFile.restrict_atoms(p0Index)


dat=p0Gro.xyz[0,:]

In [ ]:
dashApp=dashTopViz(dat)

In [7]:
ac=gudhi.AlphaComplex(dat)
st = ac.create_simplex_tree()
skel=list(st.get_skeleton(2))
skel.sort(key=lambda s: s[1])
points = np.array([ac.get_point(i) for i in range(st.num_vertices())])

In [10]:
dm=distance.pdist(points)
skelD=[(i[0],dm[pDistIndex(dm,i[0][0],i[0][1],dat.shape[0])]) if len(i[0])==2 else (i[0],max({dm[pDistIndex(dm,i[0][k],i[0][j],dat.shape[0])] for k in range(len(i[0])) for j in range(k) })) for i in skel[len(points):]] # not true for 3simplices  
skelD.sort(key=lambda s: s[1])

In [ ]:
skelD[-1000:]

In [ ]:
len(skelD)

In [ ]:
resIndex=1
curdist=0
skelBins=[0]
alphaRes=.25
maximum=20
nextHigh=0
#binLow=0
print(len(skelD))
while resIndex < len(skelD) and curdist<maximum:
    curdist=skelD[resIndex][1]
    if curdist > nextHigh:
        skelBins.append(resIndex) # up to but not including resIndex
        nextHigh+=alphaRes
        print(skelD[resIndex][1],resIndex)
    resIndex+=1
if curdist<maximum:
    skelBins.append(len(skelD))
print(len(skelD))
print(skelBins)
newskel={k*alphaRes:[j[0] for j in skelD[skelBins[k-1]:skelBins[k]]] for k in range(1,int(maximum//alphaRes))} 

In [ ]:
newskel

In [ ]:
skelD[49781]

In [ ]:
skel[len(points):len(points)+20]

In [ ]:
skelD[:20]

In [ ]:
dgm=st.persistence()

In [ ]:
[i for i in dgm if i[0]==0 and i[1][0]<.05]

In [ ]:
len(skelD)

In [ ]:
skelD[49797:49797]

# Run Server

In [ ]:
app.run_server(mode='inline',debug=True)